In [1]:
import pandas as pd
import cdsapi
import netCDF4
from netCDF4 import num2date
import numpy as np
import os
import xarray as xr
import concurrent.futures
import asyncio
import nest_asyncio
nest_asyncio.apply()

def isFilePresent( file_name):
    file_present = os.path.isfile(file_name)
    if file_present:
        print('Alreaday present : '+file_name)
    else:
        print('file absent : '+file_name)
    return file_present


In [2]:
def download_climate_data_xarray_file_only(file_parameters):


    date = file_parameters["date"]
    latitude = file_parameters["latitude"] 
    longitude = file_parameters["longitude"] 
    area = file_parameters["area"] 
    day_night_ind = file_parameters["day_night_ind"]
    
    c = cdsapi.Client()
    
    date_split = date.split("-")
    year = date_split[0]
    month = date_split[1]
    day = date_split[2]
    
    area_str = '_'.join(str(num) for num in area)
    
    file_name = './historical_climate_data/'+year+'/weather-'+str(longitude)+'-'+str(latitude)+'-'+date+'.nc'
    if not isFilePresent(file_name):
        
        time = '12:00'
        if day_night_ind == "N":
            time = '22:00'
        c.retrieve('reanalysis-era5-single-levels',
        {
            'product_type': 'reanalysis',
            'format': 'netcdf',
            'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
            'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3',
            'soil_temperature_level_4', 'soil_type', 'total_precipitation', 'volumetric_soil_water_layer_1',
            'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4'
            ],
            'year': year,
            'month': month,
            'day': [str(day)],
            'time': [time],
            'area': area,
        },
        file_name)

    
    

In [3]:
def group_fire_data_xarray_file_only(file_name):
    file_path = "./historical_fire_data/" + file_name + ".csv"
    fire_data = pd.read_csv(file_path)
    
    filtered_fire_data = fire_data.query("`confidence` >= 75 and `type` == 0")
    grouped_fire_data = filtered_fire_data.groupby(['acq_date', 'daynight']).agg({'latitude':'mean','longitude':'mean','brightness':'mean','confidence':'mean', 'frp':'mean', 'bright_t31':'mean'}).reset_index()

    #grouped_fire_data.apply(apply_on_each_group)

    result_df = pd.DataFrame({'longitude': pd.Series(dtype='float'),
                   'latitude': pd.Series(dtype='float'),
                   'date': pd.Series(dtype='str'),       
                   'daynight': pd.Series(dtype='str'),                   
                   '10m_u_component_of_wind': pd.Series(dtype='float'),
                   '10m_v_component_of_wind': pd.Series(dtype='float'),
                   '2m_temperature': pd.Series(dtype='float'),
                   'soil_temperature_level_1': pd.Series(dtype='float'),
                   'soil_temperature_level_2': pd.Series(dtype='float'),
                   'soil_temperature_level_3': pd.Series(dtype='float'),
                   'soil_temperature_level_4': pd.Series(dtype='float'),
                   'soil_type': pd.Series(dtype='float'),
                   'total_precipitation': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_1': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_2': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_3': pd.Series(dtype='float'),
                   'volumetric_soil_water_layer_4': pd.Series(dtype='float')})
    
    #print(grouped_fire_data)
    params = []
    tasks = []
    for modis_data_index, modis_row in grouped_fire_data.iterrows():
        #print(modis_row['acq_date'])
        date_value = modis_row['acq_date']
        confidence = modis_row['confidence']
        latitude = modis_row['latitude']
        longitude = modis_row['longitude']
        day_night_ind = modis_row['daynight']
        #print(type(confidence))
        if confidence > 75:
            north_coord = latitude+0.25
            south_coord = latitude-0.25
            west_coord = longitude-0.25
            east_coord = longitude+0.25
            area = [north_coord, west_coord, south_coord, east_coord]
            #print(area)

            file_paramaters = {
                "date" :date_value, 
                "latitude" : latitude, 
                "longitude" : longitude, 
                "area": area, 
                "day_night_ind": day_night_ind
            }
            params.append(file_paramaters)
            #asyncio.run(download_climate_data_xarray_file_only(date_value, latitude, longitude, area, day_night_ind))

    print(len(params))
    return params
    #for file_paramaters in params:
    #    task = asyncio.create_task(download_climate_data_xarray_file_only(file_paramaters))
    #    tasks.append(task)

    # Fire and forget the tasks
    #await asyncio.gather(*tasks)
    
    #result_filename = "./historical_climate_data/csv/"+file_name+".csv"
    #result_df.to_csv(result_filename)
    #return result_filename
    

In [4]:
params = group_fire_data_xarray_file_only("canada_2010")

490


In [5]:

# Create a ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    # Submit download_data function for each variable
    futures = [executor.submit(download_climate_data_xarray_file_only, variable) for variable in params]

    # Wait for all tasks to complete
    for future in concurrent.futures.as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print(f"An error occurred: {e}")

Alreaday present : ./historical_climate_data/2010/weather--115.76191666666666-54.21706666666666-2010-01-09.ncAlreaday present : ./historical_climate_data/2010/weather--117.72398181818181-54.218413636363636-2010-01-08.nc
Alreaday present : ./historical_climate_data/2010/weather--115.6795-54.3921-2010-01-10.nc

Alreaday present : ./historical_climate_data/2010/weather--117.40415555555556-53.859-2010-01-12.nc
Alreaday present : ./historical_climate_data/2010/weather--115.8244-52.6805-2010-01-07.nc
Alreaday present : ./historical_climate_data/2010/weather--117.51993333333333-54.111266666666666-2010-01-10.nc
Alreaday present : ./historical_climate_data/2010/weather--116.30394444444444-54.436077777777776-2010-01-14.nc
Alreaday present : ./historical_climate_data/2010/weather--118.28653333333334-53.6035380952381-2010-01-13.nc
Alreaday present : ./historical_climate_data/2010/weather--118.20855-53.81595-2010-01-17.nc
Alreaday present : ./historical_climate_data/2010/weather--115.8438-52.6537-2